<a href="https://colab.research.google.com/github/morwald/ada_project/blob/main/speakers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports of libraries and additional datasets

In [1]:
# Import epfl google drive!
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Change to shared project drive
%cd /content/drive/Shareddrives/ADA-project

/content/drive/Shareddrives/ADA-project


In [93]:
import pandas as pd
import numpy as np
import bz2
import json

from tqdm import tqdm
from collections import Counter

In [4]:
# Speaker attributes in df
df = pd.read_parquet('Project datasets/speaker_attributes.parquet')
df.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [ ]:
!python Project\ datasets/query_wikidata_dump.py --input /content/drive/MyDrive/wikidata-20211004-all.json.gz  --output speakers.json

Traceback (most recent call last):
  File "Project datasets/query_wikidata_dump.py", line 190, in <module>
    main()
  File "Project datasets/query_wikidata_dump.py", line 22, in main
    with gzip.open(args.input, 'rb') as s_file:
  File "/usr/lib/python3.7/gzip.py", line 58, in open
    binary_file = GzipFile(filename, gz_mode, compresslevel)
  File "/usr/lib/python3.7/gzip.py", line 168, in __init__
    fileobj = self.myfileobj = builtins.open(filename, mode or 'rb')
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/wikidata-20211004-all.json.gz'


##Cleaning 

### Plan
*   Descriptive statistics
*   types
*   Missing data
*   distribution (histograms)
*   outliers
*   duplicates
*  inconsistent data (wrong format etc)
*   correlation between parameters
*  filter columns that we don't need
*  maybe create additional ones
*  standardize ?

In [ ]:
#Percentage of lines with null gender : 
df[df.gender.isnull()].size / df.size

0.21536937853557775

In [5]:
#Remove null gender : 
df_2 = df.drop(df[df.gender.isnull()].index)
df_2.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [6]:
#Duplicates
duplicates = df_2[df_2.duplicated(subset='id', keep='first')] 
duplicates.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion


In [7]:
#Remove unncessary columns
df_3 = df_2.drop(['US_congress_bio_ID', 'party', 'candidacy'], axis=1) 
# drop also aliases, date_of_birth, ethnic_group, label, religion ??

In [ ]:
#Percentage of lines with null nationality : 
df_3[df_3.nationality.isnull()].size / df.size

0.2547805698797292

In [29]:
#Percentage of lines with null academic degree : 
df_3[df_3.academic_degree.isnull()].size / df.size

0.5160971517055966

In [30]:
#Remove academic group column as it has above 50% of null values : 
df_3 = df_3.drop('academic_degree', axis=1)

In [ ]:
#Percentage of lines with null date of birth : 
df_3[df_3.date_of_birth.isnull()].size / df.size

0.20053990837657457

In [ ]:
#Percentage of lines with null ethnic_group : 
df_3[df_3.ethnic_group.isnull()].size / df.size

0.6163618055294064

In [8]:
#Remove ethnic group column as it has above 50% of null values : 
df_3 = df_3.drop('ethnic_group', axis=1)

In [ ]:
#Percentage of lines with null label : 
df_3[df_3.label.isnull()].size / df.size

0.03188346648106557

In [ ]:
#Percentage of lines with null religion : 
df_3[df_3.religion.isnull()].size / df.size

0.5596987302277541

In [9]:
#Remove religion column as it has above 50% of null values : 
df_3 = df_3.drop('religion', axis=1)

In [28]:
df_3[df_3.aliases.isnull()].size / df.size

0.443215888667022

## Retrieve quotebank wikidata labels 


In [10]:
labels = pd.read_csv('Project datasets/wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')
labels.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


In [ ]:
labels.Label.isnull().count()/labels.size

0.5

In [11]:
#labels = labels.drop(labels[labels.Label.isnull()].index)
labels.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


In [ ]:
labels.loc['Q6581097']['Label']

'male'

In [ ]:
#Replacing gender QUIDs by value
#for index, row in df_3.iterrows():
#  quid = row.loc['gender'][0]
#  row.loc['gender'] = labels.loc[quid]['Label']
#--> Not useful, replace only the ones needed

###Test with the 2019 New York Times Data

In [12]:
df_quotesNY = pd.read_json('quotes-2019-nytimes.json', lines=True)
df_quotesNY.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E
2,2019-05-09-055187,It makes it much more difficult for him to mak...,None,[],2019-05-09 18:11:29,1,"[[None, 0.6493], [President Bill Clinton, 0.27...",[http://mobile.nytimes.com/2019/05/09/world/as...,E
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E
4,2019-01-04-001792,A Pile of Leaves.,None,[],2019-01-04 10:00:07,1,"[[None, 0.8737], [Jason Fulford, 0.1263]]",[https://www.nytimes.com/2019/01/04/books/revi...,E


In [13]:
#Filter null speaker 
df_quotesNY = df_quotesNY.drop(df_quotesNY[df_quotesNY.speaker == 'None'].index)
df_quotesNY.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E
5,2019-08-15-002017,A Senator we can call our own.,Tom Rath,[Q7817334],2019-08-15 22:36:33,1,"[[Tom Rath, 0.7598], [None, 0.1993], [Warren R...",[http://www.nytimes.com/2019/08/15/us/politics...,E
8,2019-07-22-032883,"It's a success, a relief and a technical feat,",Florence Parly,[Q3074013],2019-07-22 02:37:50,21,"[[Florence Parly, 0.9262], [None, 0.0738]]",[http://www.breitbart.com/news/french-submarin...,E


In [ ]:
speaker_genders = df_3['gender']
speaker_genders.value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


[Q6581097]              5418464
[Q6581072]              1684170
[Q1052281]                  887
[Q48270]                    307
[Q2449503]                  228
                         ...   
[Q6581097, Q6581072]          1
[Q179294, Q6581097]           1
[Q179294, Q6581097]           1
[Q179294, Q6581097]           1
[Q6581097, Q6581072]          1
Name: gender, Length: 1411, dtype: int64

In [ ]:
print(labels.loc['Q1052281']['Label'],
labels.loc['Q48270']['Label'],
labels.loc['Q2449503']['Label'])

transgender female non-binary transgender male


In [22]:
print(df_3.loc[df_3.id=='Q16213953']['aliases'], df_3.loc[df_3.id=='Q20707104']['aliases'], df_3.loc[df_3.id=='Q43143598']['aliases'])

5959671    [James Scott Fisher]
Name: aliases, dtype: object 2624613    None
Name: aliases, dtype: object 7255341    None
Name: aliases, dtype: object


In [24]:
df_3.loc[df_3.id=='Q105756']['aliases']

3402750    [John Hoyer Updike]
Name: aliases, dtype: object

In [27]:
print(df_3.loc[df_3.id=='Q19594790']['nationality'], df_3.loc[df_3.id=='Q34975']['nationality'])

8268745    [Q145]
Name: nationality, dtype: object 5661151    [Q22]
Name: nationality, dtype: object


In [ ]:
#ALL ALIASES
#Goal : struct dictionary{key = quote_id, value = dict{key=speaker_id, value = [genders per speaker]}}

# Retrieve New York times speakers' gender, in the order of the quotations df
dict_quote_id_genders = {}

for NY_index, NY_row in df_quotesNY.iterrows(): #Loop over all quotes
  quote_id = NY_row.quoteID #Future key in dict_quote_id_genders
  id_aliases = NY_row.loc['qids'] 
  dict_genders_speaker = {} #Dict of genders per alias speaker
  for id_speaker in id_aliases:
    list_genders_speaker = [] #List of genders per alias speaker
    id_genders_speaker = df_3.loc[df_3.id==id_speaker]['gender'].to_numpy() #List of all genders ID per speaker
    for id_g in id_genders_speaker:
      gender= labels.loc[id_g]['Label'].values #Gender value
      list_genders_speaker.append(gender)
    dict_genders_speaker[id_speaker] = list_genders_speaker
    print(dict_genders_speaker)
  dict_quote_id_genders[quote_id] = dict_genders_speaker

{'Q16213953': [array(['male'], dtype=object)]}
{'Q16213953': [array(['male'], dtype=object)], 'Q20707104': [array(['male'], dtype=object)]}
{'Q16213953': [array(['male'], dtype=object)], 'Q20707104': [array(['male'], dtype=object)], 'Q43143598': [array(['male'], dtype=object)]}
{'Q16213953': [array(['male'], dtype=object)], 'Q20707104': [array(['male'], dtype=object)], 'Q43143598': [array(['male'], dtype=object)], 'Q58886302': [array(['male'], dtype=object)]}
{'Q16213953': [array(['male'], dtype=object)], 'Q20707104': [array(['male'], dtype=object)], 'Q43143598': [array(['male'], dtype=object)], 'Q58886302': [array(['male'], dtype=object)], 'Q6133913': [array(['male'], dtype=object)]}
{'Q105756': [array(['male'], dtype=object)]}
{'Q1855840': [array(['male'], dtype=object)]}
{'Q7817334': [array(['male'], dtype=object)]}
{'Q3074013': [array(['female'], dtype=object)]}
{'Q7812406': [array(['male'], dtype=object)]}
{'Q977546': [array(['male'], dtype=object)]}
{'Q50049': [array(['female'], 

KeyboardInterrupt: ignored

In [95]:

#ONLY FIRST ALIAS
#Goal : struct dictionary{key = quote_id, value = ([genders],[nationalities])}

# Retrieve quotes speakers' gender, in the order of the quotations df
# Input : df_quotes with only first qid in new column qid (instead of qids)
def gender_nat_speaker_dict(df_quotes):
  dict_quote_id_genders = {}

  for q_index, q_row in tqdm(df_quotes.head(10).iterrows(), total=df_quotes.shape[0]): #Loop over all quotes
    quote_id = q_row.quoteID #Future key in dict_quote_id_genders
    id_speaker = q_row.loc['qid'] 
    list_genders_speaker = [] #List of genders per alias speaker
    list_nat_speaker = [] #List of nationalities per alias speaker
    id_genders_speaker = df_3.loc[df_3.id==id_speaker]['gender'].to_numpy() #List of all genders ID per speaker
    id_nat_speaker = df_3.loc[df_3.id==id_speaker]['nationality'].to_numpy() #List of all nationalities ID per speaker
    for id_g in id_genders_speaker:
      gender= labels.loc[id_g]['Label'].values #Gender value
      list_genders_speaker.append(gender)
    for id_n in id_nat_speaker:
      if(id_n is not None):
        nat= labels.loc[id_n]['Label'].values #Nationality value
        list_nat_speaker.append(nat)
    dict_quote_id_genders[quote_id] = (list_genders_speaker, list_nat_speaker)
  return dict_quote_id_genders

In [110]:
#ONLY FIRST ALIAS DICTIONARY
#Goal : struct dictionary{key = quote_id, value = ([genders],[nationalities])}

# Retrieve quotes speakers' gender, in the order of the quotations df
# Input : df_quotes with only first qid in new column qid (instead of qids)
def list_attribute_speaker(id_speaker, attributes):
  list_attr_speaker = []
  for attr in attributes:
    list_attr = []
    id_attr_speaker = df_3.loc[df_3.id==id_speaker][attr].to_numpy() #List of all attribute ID per speaker
    for id_a in id_attr_speaker:
      if(id_a is not None):
        attr= labels.loc[id_a]['Label'].values #Attribute value
        list_attr.append(attr[0])
    list_attr_speaker.append(list_attr)
  return (id_speaker, list_attr_speaker)


In [112]:
dict_genders = {}
dict_genders = dict(df_quotesNY['qid'].head(10).apply(lambda i: list_attribute_speaker(i, ['gender', 'nationality'])).tolist())
dict_genders

{'Q105756': [['male'], ['United States of America']],
 'Q16213953': [['male'], ['United Kingdom']],
 'Q1855840': [['male'], ['United States of America']],
 'Q19594790': [['male'], ['United Kingdom']],
 'Q22686': [['male'], ['United States of America']],
 'Q3074013': [['female'], ['France']],
 'Q50049': [['female'], ['United States of America']],
 'Q7812406': [['male'], ['United States of America']],
 'Q7817334': [['male'], ['United States of America']],
 'Q977546': [['male'], ['United States of America']]}

In [74]:
#ONLY FIRST ALIAS
#Goal : add directly in df gender and nationality

# Retrieve quotes speakers' gender, in the order of the quotations df
# Input : df_quotes with only first qid in new column qid (instead of qids)
def gender_nat_speaker(df_quotes):
  df_result = df_quotes.copy()
  list_genders_speakers = [] #List of genders alias speakers
  list_nat_speakers = [] #List of nationalities alias speakers
  for q_index, q_row in df_result.iterrows(): #Loop over all quotes
    quote_id = q_row.quoteID #Future key in dict_quote_id_genders
    list_genders_speaker = [] #List of genders per alias speaker
    list_nat_speaker = [] #List of nationalities per alias speaker
    id_genders_speaker = df_3.loc[df_3.id==id_speaker]['gender'].to_numpy() #List of all genders ID per speaker
    id_nat_speaker = df_3.loc[df_3.id==id_speaker]['nationality'].to_numpy() #List of all nationalities ID per speaker
    for id_g in id_genders_speaker:
      gender= labels.loc[id_g]['Label'].values #Gender value
      list_genders_speaker.append(gender)
    for id_n in id_nat_speaker:
      if(id_n is not None):
        nat= labels.loc[id_n]['Label'].values #Nationality value
        list_nat_speaker.append(nat)
    list_genders_speakers.append(list_genders_speaker)
    list_nat_speakers.append(list_nat_speaker)
  df_result.insert(4, "gender", list_genders_speakers)
  df_result.insert(5, 'nationality', list_nat_speakers)
  return df_result

In [65]:
def get_single_qid(df_quotes):

  df_copy =  df_quotes.copy()
  df_result =  df_quotes.copy()
  for idx,row in df_copy.iterrows():  
    id_speaker = row.loc['qids'][0] 
    df_result.at[idx,'qids'] = id_speaker

  df_result = df_result.rename(columns={"qids": "qid"})
  return df_result


### Old examples

In [ ]:
path_to_file = '/content/drive/MyDrive/speakers.json' 

with bz2.open(path_to_file, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) # loading a sample
    print(instance) 

In [ ]:
# Loading of the quotebank dataset

path_to_file = '/content/drive/MyDrive/Quotebank/quotes-2020.json.bz2' 

with bz2.open(path_to_file, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) # loading a sample
    print(instance) 